In [61]:
#Mount Google Drive

from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [62]:
#Importing required libraries for the NLP 2 Capstone Project

import pandas as pd
#!pip install pycaret shap
from pycaret.classification import * 
from sklearn.feature_extraction.text import TfidfVectorizer

data_path = r'/content/drive/My Drive/Colab Notebooks/GL Capstone Project/'
file_name = 'capproj1.csv'
output_model_file_name = 'TFID_Bestmodel_Lr'

In [63]:
#Import data

tfid = pd.read_csv(data_path + file_name)
tfid = tfid.drop(columns='Unnamed: 0', axis=1)
tfid.head()

,Countries,Accident Level,Description
0,Country_01,I,removing drill rod jumbo maintenance superviso...
1,Country_02,I,activation sodium sulphide pump piping uncoupl...
2,Country_01,I,sub station milpo located level collaborator e...
3,Country_01,I,approximately nv cx ob personnel begins task u...
4,Country_01,IV,approximately circumstances mechanics anthony ...


In [64]:
# BoW-TF:IDF Embedding

tfidf_vectorizer = TfidfVectorizer(min_df=.02, max_df=.7, ngram_range=[1,3])

%time tfid_model = tfidf_vectorizer.fit_transform(tfid['Description'])
display("Bow-TF:IDF :", tfid_model.shape)
tfid_model_data = pd.DataFrame(tfid_model.toarray(), columns=tfidf_vectorizer.get_feature_names(), index=tfid.index)
tfid_model_data.head()

CPU times: user 69.2 ms, sys: 3.74 ms, total: 73 ms
Wall time: 72.8 ms


'Bow-TF:IDF :'

(425, 340)

,access,accident,accident employee,acid,activities,activity,air,allergic,allergic reaction,another,...,weight kg,went,without,wooden,work,worker,workers,workshop,wound,zinc
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.176768,0.000000,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.241787,0.0,0.0,0.0,0.000000,0.159894,0.0,0.0,0.0,0.0


In [65]:

tfid_model_data_prep = pd.concat([tfid, tfid_model_data], axis = 1)
tfid_model_data_prep.drop(columns=['Countries', 'Description'], inplace = True)
print(tfid_model_data_prep.shape)
tfid_model_data_prep.head()

(425, 341)


,Accident Level,access,accident,accident employee,acid,activities,activity,air,allergic,allergic reaction,...,weight kg,went,without,wooden,work,worker,workers,workshop,wound,zinc
0,I,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0
1,I,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0
2,I,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.176768,0.000000,0.0,0.0,0.0,0.0
3,I,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0
4,IV,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.241787,0.0,0.0,0.0,0.000000,0.159894,0.0,0.0,0.0,0.0


In [66]:
tfid_model_data_prep_columns = tfid_model_data_prep.columns
print(tfid_model_data_prep_columns)

res = [sub for sub in tfid_model_data_prep_columns if sub.isalpha()]
res.append('Accident Level')

Index(['Accident Level', 'access', 'accident', 'accident employee', 'acid',
       'activities', 'activity', 'air', 'allergic', 'allergic reaction',
       ...
       'weight kg', 'went', 'without', 'wooden', 'work', 'worker', 'workers',
       'workshop', 'wound', 'zinc'],
      dtype='object', length=341)


In [67]:
tfid_model_data_prep = tfid_model_data_prep.drop(columns=[col for col in tfid_model_data_prep if col not in res])
print(tfid_model_data_prep.shape)
tfid_model_data_prep.head()

(425, 305)


,Accident Level,access,accident,acid,activities,activity,air,allergic,another,approx,...,weight,went,without,wooden,work,worker,workers,workshop,wound,zinc
0,I,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0
1,I,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0
2,I,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00000,0.0,0.0,0.0,0.176768,0.000000,0.0,0.0,0.0,0.0
3,I,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0
4,IV,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.19724,0.0,0.0,0.0,0.000000,0.159894,0.0,0.0,0.0,0.0


In [68]:
%time pce_1 = setup(data = tfid_model_data_prep, target = 'Accident Level', session_id = 5, train_size = 0.85, silent=True)

,Description,Value
0,session_id,5
1,Target,Accident Level
2,Target Type,Multiclass
3,Label Encoded,"I: 0, II: 1, III: 2, IV: 3, V: 4"
4,Original Data,"(425, 305)"
5,Missing Values,False
6,Numeric Features,304
7,Categorical Features,0
8,Ordinal Features,False
9,High Cardinality Features,False


CPU times: user 1.7 s, sys: 138 ms, total: 1.84 s
Wall time: 1.73 s


In [69]:
%time compare_models()
#From below Logistic Regression provides the best accuracy

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
lr,Logistic Regression,0.7424,0.4645,0.2150,0.5513,0.6327,0.0000,0.0000,0.048
dummy,Dummy Classifier,0.7424,0.3500,0.2150,0.5513,0.6327,0.0000,0.0000,0.017
rf,Random Forest Classifier,0.7396,0.4056,0.2143,0.5539,0.6333,0.0096,0.0177,0.564
et,Extra Trees Classifier,0.7369,0.4141,0.2135,0.5549,0.6330,0.0114,0.0158,0.522
ridge,Ridge Classifier,0.7340,0.0000,0.2170,0.5622,0.6357,0.0277,0.0483,0.023
knn,K Neighbors Classifier,0.7285,0.4305,0.2111,0.5502,0.6268,-0.0099,-0.0150,0.126
lightgbm,Light Gradient Boosting Machine,0.7119,0.4207,0.2224,0.5755,0.6331,0.0391,0.0585,0.175
ada,Ada Boost Classifier,0.6842,0.3103,0.2241,0.5721,0.6171,0.0283,0.0292,0.151
svm,SVM - Linear Kernel,0.6787,0.0000,0.2544,0.6062,0.6393,0.1262,0.1326,0.071
nb,Naive Bayes,0.6732,0.3609,0.2578,0.6042,0.6331,0.0807,0.0854,0.025


CPU times: user 5.34 s, sys: 332 ms, total: 5.67 s
Wall time: 36.5 s


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=1000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=5, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [70]:
#using Logistic Regression model
%time logreg = create_model('lr')
tuned_logreg = tune_model(logreg)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.7297,0.5684,0.2000,0.5325,0.6157,0.0,0.0
1,0.7500,0.0000,0.2500,0.5625,0.6429,0.0,0.0
2,0.7500,0.0000,0.2500,0.5625,0.6429,0.0,0.0
3,0.7500,0.0000,0.2500,0.5625,0.6429,0.0,0.0
4,0.7500,0.7779,0.2000,0.5625,0.6429,0.0,0.0
5,0.7500,0.5591,0.2000,0.5625,0.6429,0.0,0.0
6,0.7500,0.6636,0.2000,0.5625,0.6429,0.0,0.0
7,0.7500,0.5718,0.2000,0.5625,0.6429,0.0,0.0
8,0.7222,0.7008,0.2000,0.5216,0.6057,0.0,0.0


In [71]:
evaluate_model(tuned_logreg)

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Hyperparameters', 'param…

In [72]:
#predict model
predict_model(tuned_logreg, tfid_model_data_prep.tail()) 
#see Label columns on the right end, it shows classficaition results for the records

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Logistic Regression,0,0,0,0,0,0,0


,Accident Level,access,accident,acid,activities,activity,air,allergic,another,approx,...,without,wooden,work,worker,workers,workshop,wound,zinc,Label,Score
420,I,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,I,0.6906
421,I,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,I,0.8342
422,I,0.0,0.0,0.0,0.0,0.187549,0.0,0.0,0.0000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,I,0.8075
423,I,0.0,0.0,0.0,0.0,0.185333,0.0,0.0,0.2871,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,I,0.7241
424,I,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,I,0.7916


In [73]:
#pickling the model for future use
save_model(tuned_logreg, model_name=data_path + output_model_file_name) 

Transformation Pipeline and Model Successfully Saved


(Pipeline(memory=None,
          steps=[('dtypes',
                  DataTypes_Auto_infer(categorical_features=[],
                                       display_types=False, features_todrop=[],
                                       id_columns=[],
                                       ml_usecase='classification',
                                       numerical_features=[],
                                       target='Accident Level',
                                       time_features=[])),
                 ('imputer',
                  Simple_Imputer(categorical_strategy='not_available',
                                 fill_value_categorical=None,
                                 fill_value_numerical=None,
                                 numer...
                 ('feature_select', 'passthrough'), ('fix_multi', 'passthrough'),
                 ('dfs', 'passthrough'), ('pca', 'passthrough'),
                 ['trained_model',
                  LogisticRegression(C=1.458, class_